In [82]:
# import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
import scipy.special as special
import time as time
import datetime as dt
import os

from bs4 import BeautifulSoup
import requests 

Yes24 Bestseller Data in Dataframe

In [83]:
# get bestseller links
yes24_url = 'http://www.yes24.com/24/Category/BestSeller' 
result = requests.get(yes24_url)
raw_html = BeautifulSoup(result.text, 'html.parser')

a_html = raw_html.find('ol')
list_yes24_bestseller_data = a_html.find_all('p', class_='copy')

# create list of all bestseller urls
yes24_bestseller_urls = [] 
for i in list_yes24_bestseller_data:
    urls_in_html = i.find('a')
    yes24_bestseller_urls.append('http://www.yes24.com/' + urls_in_html.get('href'))

In [84]:
# set frame for data
data = []

# get data for each best seller, create 2D array
for url in yes24_bestseller_urls: 
    result = requests.get(url) # get html
    raw_html = BeautifulSoup(result.text, 'html.parser')

    title = raw_html.find(name='h2', class_='gd_name').string

    author_wrapper = raw_html.find(name='span', class_='gd_auth')
    author = author_wrapper.find('a').string

    publisher_wrapper = raw_html.find(name='span', class_='gd_pub')
    publisher = publisher_wrapper.find('a').string

    published_date_str = raw_html.find(name='span', class_='gd_date').string
    published_date_num = published_date_str.translate({ord(i): None for i in '월년일'})

    # 주의) 가급적이면 원하는 값에'만' 해당 되는 class, tag, id 등으로 찾기 b/c html data가 바뀌면서 가져오는 값도 바뀌게 되어짐  
    origin_wrapper = raw_html.find(name='span', class_='gd_orgin') # for loop, 각 bestseller에 대해 출력
    if origin_wrapper != None:
        origin_wrapper = origin_wrapper.find('a').string

    yes24_review_score_tag = raw_html.find(name='em', class_='yes_b') # for loop, 각 bestseller에 대해 출력
    if yes24_review_score_tag is not None:
        yes24_review_score = yes24_review_score_tag.string
    
    yes24_review_number_tag = raw_html.find(name='span', class_='gd_reviewCount').a.em
    if yes24_review_number_tag is not None:
        yes24_review_number = yes24_review_number_tag.string

    yes24_fixed_price = raw_html.find_all(name='em', class_='yes_m')[0]
    yes24_fixed_price = yes24_fixed_price.string
    yes24_selling_price = raw_html.find_all(name='em', class_='yes_m')[1]
    yes24_selling_price = yes24_selling_price.string

    ISBN13_parent_html = raw_html.find(name='th', class_='txt', string='ISBN13').parent
    ISBN13 = ISBN13_parent_html.find(name='td', class_='txt lastCol').string

    ISBN10_parent_html = raw_html.find(name='th', class_='txt', string='ISBN10').parent
    ISBN10 = ISBN10_parent_html.find(name='td', class_='txt lastCol').string

    yes24_sales_text = raw_html.find(name ='span', class_='gd_sellNum')
    yes24_sales_text = yes24_sales_text.text
    yes24_sales_index = ''
    for i in yes24_sales_text:
        if i.isnumeric() == True:
            yes24_sales_index = yes24_sales_index + i

    genre_list = raw_html.find(name='em', class_='bl_dot bgYUI').parent
    yes24_final_genre_name = genre_list.find_all('a')[-1].string

    yes24_bestseller_data = [title, author, publisher, published_date_num, origin_wrapper, yes24_review_score, yes24_review_number, yes24_fixed_price, yes24_selling_price, ISBN13, ISBN10, yes24_sales_index, yes24_final_genre_name]
    data.append(yes24_bestseller_data)

In [85]:
# create dataframe with values
columns=['Title', 'Author', 'Publisher', 'Published_date', 'Origin', 'Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Fixed_price', 'Yes24_Selling_price', 'ISBN13', 'ISBN10', 'Yes24_sales_index', 'Yes24_Final_genre_name'] # change to all caps or all lowercase
number_bestsellers = len(yes24_bestseller_urls)
rank = np.arange(1,number_bestsellers+1)
yes24_dataframe = pd.DataFrame(data=data, index=rank, columns=columns)
yes24_dataframe['Published_date'] = pd.to_datetime(yes24_dataframe['Published_date'], format='%Y %m %d')

Aladin Bestseller Data in CSV

In [86]:
# Get excel file from Aladin, 데이트 타임 활용하기
Aladin_URL = 'https://www.aladin.co.kr/shop/common/wbest_excel.aspx?BestType=Bestseller&BranchType=1&CID=0&Year=2023&Month=2&Week=3' # 링크가 매주 바뀜 

req = requests.get(Aladin_URL) # 200이면 온라인

Aladin_excel_name = '알라딘_주간+베스트_국내도서_2023년2월3주_20230226.csv' # 일반화 & edit later 

with open(Aladin_excel_name, 'wb') as f: # write, binary, with 사용해서 효율적
    for chunk in req.iter_content(chunk_size=9000): # 실제로 파일을 열어서 작성
        if chunk:
            f.write(chunk)

os.rename(r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\알라딘_주간+베스트_국내도서_2023년2월3주_20230226.csv', r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\aladin_bestseller_data.csv')

Kyobo Bestseller Data in CSV

In [87]:
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [88]:
# Get excel file from Kyobobook with Selenium
import shutil

Kyobo_URL = "https://product.kyobobook.co.kr/bestseller/online?period=001#?page=1&per=20&ymw=&period=001&saleCmdtClstCode=&dsplDvsnCode=000&dsplTrgtDvsnCode=001&saleCmdtDsplDvsnCode="

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # headless 해결해보기

driver.get(url=Kyobo_URL)
driver.implicitly_wait(time_to_wait=3)

download_excel_button = driver.find_element(By.XPATH, '//*[@id="excel_btn"]')
download_excel_button.click() 

time.sleep(10)

driver.close()

shutil.move(r'C:\Users\seohy\Downloads\교보문고_온라인_베스트셀러_상품리스트.xlsx', r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\교보문고_온라인_베스트셀러_상품리스트.xlsx') # 파일 이름에 날짜 붙이기

os.rename(r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\교보문고_온라인_베스트셀러_상품리스트.xlsx', r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\kyobo_bestseller_data.xlsx')


Bring All Three Files Together 

In [89]:
aladin_path = r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\aladin_bestseller_data.csv'
kyobo_path = r'H:\내 드라이브\파이썬\Analyze_Bestseller_Project\kyobo_bestseller_data.xlsx'

aladin_dataframe = pd.read_csv(aladin_path)
kyobo_dataframe = pd.read_excel(kyobo_path)

c:\Users\seohy\AppData\Local\Programs\Python\Python311\Lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [90]:
# Clean and reorganize yes24 dataframe
yes24_dataframe['ISBN13'] = yes24_dataframe['ISBN13'].astype(str)
yes24_dataframe = yes24_dataframe.set_index('ISBN13')

yes24_dataframe_columns_new_order = ['Title', 'Author', 'Publisher', 'Published_date', 'Origin', 'Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Fixed_price', 'Yes24_Selling_price', 'Yes24_sales_index', 'Yes24_Final_genre_name']
yes24_dataframe = yes24_dataframe[yes24_dataframe_columns_new_order]


In [91]:
# Clean and organize Aladin Dataframe
aladin_dataframe = aladin_dataframe.drop(index=1000)

aladin_dataframe['ISBN13'] = aladin_dataframe['ISBN13'].astype(str)
aladin_dataframe['ISBN13'] = aladin_dataframe['ISBN13'].str.replace('.0', '', regex=False) # .str으로 value 선택
aladin_dataframe = aladin_dataframe.set_index('ISBN13')

aladin_dataframe = aladin_dataframe.rename(columns = {'정가': 'Aladin_Fixed_price', '판매가': 'Aladin_Selling_price', '마일리지': 'Aladin_Mileage', '세일즈포인트': 'Aladin_Sales_Point'})
picked_aladin_dataframe = aladin_dataframe[['Aladin_Fixed_price', 'Aladin_Selling_price', 'Aladin_Mileage', 'Aladin_Sales_Point']]
picked_aladin_dataframe


,Aladin_Fixed_price,Aladin_Selling_price,Aladin_Mileage,Aladin_Sales_Point
ISBN13,,,,
9791169794930,"20,000","18,000","1,000점",356655.0
9791168473690,"7,200","6,480",360점,541195.0
9791197871269,"22,000","19,800","1,100점",104470.0
9791198013026,"18,000","16,200",900점,154420.0
9791197389450,"17,000","15,300",850점,120130.0
...,...,...,...,...
9791185578491,"20,000","18,000","1,000점",20091.0
9791168125001,"22,000","19,800","1,100점",18730.0
9788954753937,"15,000","13,500",750점,18281.0


In [92]:
# Clean and organize Kyobo Dataframe
kyobo_dataframe = kyobo_dataframe.rename(columns = {'정가': 'Kyobo_Fixed_price', '판매가': 'Kyobo_Selling_price', '적립예정포인트': 'Kyobo_Sales_Point', '분야': 'Kyobo_Genre'})
picked_kyobo_dataframe = kyobo_dataframe[['Kyobo_Fixed_price', 'Kyobo_Selling_price', 'Kyobo_Sales_Point', 'Kyobo_Genre']]
picked_kyobo_dataframe

,Kyobo_Fixed_price,Kyobo_Selling_price,Kyobo_Sales_Point,Kyobo_Genre
0,"18,000","16,200",900,자기계발
1,"7,200","6,480",360,자기계발
2,"19,000","17,100",950,경제/경영
3,"17,000","15,300",850,자기계발
4,"19,000","19,000",570,시그니처 향
...,...,...,...,...
980,"10,800","10,800",540,NaN
981,"5,500","5,500",270,NaN
982,"4,500","4,050",225,NaN
983,"4,000","4,000",120,시그니처 향


In [93]:
aladin_filter = []
for i in aladin_dataframe.index.tolist():
    truth = i in yes24_dataframe.index.tolist()
    aladin_filter.append(truth)
picked_aladin_dataframe = picked_aladin_dataframe[aladin_filter]


# kyobo_filter = [] # Index is not ISBN
# for i in kyobo_dataframe['상품명'].tolist():
#     truth = i in yes24_dataframe['Title'].tolist()
#    kyobo_filter.append(truth)
# picked_kyobo_dataframe = picked_kyobo_dataframe[kyobo_filter]

adding_frames = [yes24_dataframe, picked_aladin_dataframe]
total_bestseller_data = pd.concat(adding_frames, axis=1)

total_bestseller_data.to_excel('total_bestseller_data.xlsx', encoding='CP949')


c:\Users\seohy\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\util\_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)


In [109]:
total_bestseller_data = pd.read_excel(r"H:\내 드라이브\파이썬\Analyze_Bestseller_Project\total_bestseller_data.xlsx")

In [112]:
# getting data complete, now need to clean data
total_bestseller_data.dtypes

# how to do this more cleanly/at once?
total_bestseller_data['Yes24_Fixed_price'] = total_bestseller_data['Yes24_Fixed_price'].str.replace('원', '')
total_bestseller_data['Yes24_Fixed_price'] = total_bestseller_data['Yes24_Fixed_price'].str.replace(',', '') 
total_bestseller_data['Yes24_Selling_price'] = total_bestseller_data['Yes24_Selling_price'].str.replace(',', '')
total_bestseller_data['Yes24_Review_score'] = total_bestseller_data['Yes24_Review_score'].str.replace(',', '') 
total_bestseller_data['Aladin_Fixed_price'] = total_bestseller_data['Aladin_Fixed_price'].str.replace(',', '')
total_bestseller_data['Aladin_Selling_price'] = total_bestseller_data['Aladin_Selling_price'].str.replace(',', '')
total_bestseller_data['Aladin_Mileage'] = total_bestseller_data['Aladin_Mileage'].str.replace('점', '')
total_bestseller_data['Aladin_Mileage'] = total_bestseller_data['Aladin_Mileage'].str.replace(',', '')


# using loc for index, column
# total_bestseller_data.loc['9788936448370', 'Yes24_Review_score'] = 0

# warning, meaning?
# total_bestseller_data[total_bestseller_data.eq("14,400").any(1)]

# 그냥 'yes24_review_score'에서 all comma 지우기?
# dataframe 전체에서 comma 지우는 방법은 없을까?

# need to check if there was errors while gathering data if there are repeated erorrs and weird data in dataframe


In [113]:
total_bestseller_data[['Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Selling_price', 'Yes24_sales_index', 'Aladin_Fixed_price', 'Aladin_Selling_price']] = total_bestseller_data[['Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Selling_price', 'Yes24_sales_index', 'Aladin_Fixed_price', 'Aladin_Selling_price']].astype(float)

total_bestseller_data[['Yes24_Fixed_price', 'Aladin_Mileage']] = total_bestseller_data[['Yes24_Fixed_price', 'Aladin_Mileage']].astype(float)

#total_bestseller_data

In [115]:
total_bestseller_data_group = total_bestseller_data.groupby('Yes24_Final_genre_name')
total_bestseller_data_group[['Yes24_Review_score', 'Yes24_Review_number', 'Yes24_Fixed_price', 'Yes24_Selling_price', 'Yes24_sales_index']].agg(['mean', 'median']) # guessing that if in integer form the code automatically calculates it as a number even though it is a string
# there are weird results below, need to go back to raw data and see if there are any problems with it

Yes24_Review_score           Yes24_Review_number  \
                                     mean    median                mean   
Yes24_Final_genre_name                                                    
1-2학년 애니메이션/영화드라마 원작         11700.000000  11700.00           80.000000   
1-2학년 창작동화                   49500.000000  49500.00           19.000000   
1-2학년 학습만화                    4206.666667     10.00           29.000000   
SQL Server                       9.100000      9.10           52.000000   
건강에세이/건강기타                       9.600000      9.60           50.000000   
경영전략/경영혁신                        9.300000      9.30           19.000000   
경제전망                            10.000000     10.00            7.000000   
교양으로 읽는 인문                       9.850000      9.85           32.000000   
네이버 연재                       14400.000000  14400.00          207.000000   
노년 / 죽음                          9.600000      9.60           75.000000   
독서교육                             9.900000      9.90           49.000000   
독해/문법/RC                         9.700000      9.70           80.000000   
명사/연예인 에세이                       9.400000      9.40           26.000000   
부동산/경매                           7.400000      7.40            3.000000   
생물학                              9.100000      9.10          245.000000   
성공학/경력관리                         9.550000      9.55          206.500000   
세계사/세계문화                        10.000000     10.00            2.000000   
스포츠                              9.700000      9.70           66.333333   
연애/사랑소설                          9.200000      9.20          137.000000   
재테크일반                            9.400000      9.40          213.000000   
정치/외교학 일반                        9.600000      9.60           20.000000   
주식/증권                           10.000000     10.00            3.000000   
처세술/삶의 자세                        9.500000      9.55           51.250000   
청해/LC                            9.600000      9.60           77.000000   
한국 장편소설                          9.433333      9.40          252.333333   

                              Yes24_Fixed_price          Yes24_Selling_price  \
                       median              mean   median                mean   
Yes24_Final_genre_name                                                         
1-2학년 애니메이션/영화드라마 원작     80.0      13000.000000  13000.0             11700.0   
1-2학년 창작동화               19.0      11000.000000  11000.0              9900.0   
1-2학년 학습만화               13.0      13933.333333  14000.0             12540.0   
SQL Server               52.0      18000.000000  18000.0             18000.0   
건강에세이/건강기타               50.0      19000.000000  19000.0             17100.0   
경영전략/경영혁신                19.0      13000.000000  13000.0             11700.0   
경제전망                      7.0      19000.000000  19000.0             17100.0   
교양으로 읽는 인문               32.0      25400.000000  25400.0             22860.0   
네이버 연재                  207.0      16000.000000  16000.0             14400.0   
노년 / 죽음                  75.0      17200.000000  17200.0             15480.0   
독서교육                     49.0      18800.000000  18800.0             16920.0   
독해/문법/RC                 80.0      17800.000000  17800.0             16020.0   
명사/연예인 에세이               26.0      17000.000000  17000.0             15300.0   
부동산/경매                    3.0      20000.000000  20000.0             18000.0   
생물학                     245.0      17000.000000  17000.0             15300.0   
성공학/경력관리                206.5      18000.000000  18000.0             16200.0   
세계사/세계문화                  2.0      19800.000000  19800.0             17820.0   
스포츠                      67.0       9166.666667   7000.0              8250.0   
연애/사랑소설                 137.0      14000.000000  14000.0             12600.0   
재테크일반                   213.0      17000.000000  17000.0             15300.0   
정치/외교학 일반                20.0      19000.000000  19000.0             17100.0   
주식